# Dataset for synthesizer sound

</br>

- This section is designed to create a synthesizer dataset using a random sound sample (wav).

- It assumes that the raw sound sample file name includes the artist name, instrument, and a descriptive word for the sound texture.

- The 'Creating CSV from wav file' code section explains how to extract the file name from the wav file and generate a CSV file.

- The 'Wav to JSON' code section describes how to create a JSON file from the CSV. The information in the JSON file will be used alongside the wav file when it is input into the deep learning model (In this case, we used the LAION AI - CLAP Model).


## Creating CSV from wav file

In [13]:
import os
import csv
import shutil

def parse_filename(filename):
    split_name = filename.split('_')
    artist = split_name[0].lower()
    instrument = None
    key = None
    is_one_shot = False
    description = []
    etc = []

    for part in split_name[1:]:
        if part.lower() in ['lead', 'pluck', 'bass']:
            instrument = part.lower()
        elif 'one' in part.lower() and 'shot' in part.lower():
            is_one_shot = True
        elif any(c.isdigit() for c in part) or part.lower() in ['c', 'd', 'e', 'f', 'g', 'a', 'b', 'maj', 'min']:
            key = part.lower()
        else:
            etc.append(part)

    if instrument is None:
        file_without_ext = os.path.splitext(filename)[0]
        last_part = file_without_ext.split('_')[-1].lower()
        if last_part in ['lead', 'pluck', 'bass']:
            instrument = last_part
            if instrument in etc:
                etc.remove(instrument)

    etc = ' '.join(etc).replace('.wav', '')
    
    exclude_words = ['synth', 'c', 'd', 'e', 'f', 'g', 'a', 'b', 
                     'C','D','E','F','G','A','B',
                     'cmaj', 'dmaj', 'emaj', 'fmaj', 'gmaj', 'amaj', 'bmaj', 
                     'cmin', 'dmin', 'emin', 'fmin', 'gmin', 'amin', 'bmin', 
                     'wav', '.', 'oneshot', 'one shot', 'one_shot', '_one_shot_', 
                     'pad', 'pluck', 'lead', 'bass', 
                     'bass.wav', 'lead.wav', 'pluck.wav',
                     'c.wav', 'd.wav', 'e.wav', 'f.wav', 'g.wav', 'a.wav', 'b.wav', 
                     'c#.wav', 'd#.wav', 'e#.wav', 'f#.wav', 'g#.wav', 'a#.wav', 'b#.wav'
                     'C.wav', 'D.wav', 'E.wav', 'F.wav', 'G.wav', 'A.wav', 'B.wav',
                     'C#.wav', 'D#.wav', 'E#.wav', 'F#.wav', 'G#.wav', 'A#.wav', 'B#.wav',
                     'c', 'd', 'e', 'f', 'g', 'a', 'b', 
                     'c#', 'd#', 'e#', 'f#', 'g#', 'a#', 'b#v'
                     'C', 'D', 'E', 'F', 'G', 'A', 'B',
                     'C#', 'D#', 'E#v', 'F#', 'G#', 'A#', 'B#',
                     'cmaj.wav', 'dmaj.wav', 'emaj.wav', 'fmaj.wav', 'gmaj.wav', 'amaj.wav', 'bmaj.wav', 
                     'cmin.wav', 'dmin.wav', 'emin.wav', 'fmin.wav', 'gmin.wav', 'amin.wav', 'bmin.wav',
                     'A#maj', 'G#min', 'F#maj','E#maj','E-G.wav','C#maj','.wav','C#min','D#maj','A-B.wav','one','shot','D#min','G#maj','_F#maj_','_F#maj', 'F#maj_','Am','_C#maj']
    exclude_words = [word.lower() for word in exclude_words]
    description = ' '.join([word for word in etc.split() if word.lower() not in exclude_words and not any(char.isdigit() for char in word)])

    description = description.replace('.wav', '').lower()

    return artist, instrument, key, is_one_shot, description



In [14]:
def create_csv(csv_file_path, wav_folder_path):
    with open(csv_file_path, 'w', newline='', encoding='utf-8') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(['File_Name', 'Artist', 'Instrument', 'Etc', 'Words'])

        for file in os.listdir(wav_folder_path):
            if file.endswith('.wav'):
                artist, instrument, key, is_one_shot, description = parse_filename(file)
                etc = ' '.join([str(key), 'one_shot' if is_one_shot else ''])

                # the filename to lowercase
                file = file.lower()

                # Only write rows with no empty columns
                if all([file, artist, instrument, etc, description]):
                    csv_writer.writerow([file, artist, instrument, etc, description])


def gather_files(csv_file_path, src_folder_path, dst_folder_path):
    if not os.path.exists(dst_folder_path):
        os.makedirs(dst_folder_path)

    with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            filename = row['File_Name']
            src_file = os.path.join(src_folder_path, filename)
            dst_file = os.path.join(dst_folder_path, filename)

            if os.path.exists(src_file):
                shutil.copy2(src_file, dst_file)


csv_file_path = 'synthsoundoutput5.csv'
wav_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/soundsprayaiffelsynth"
dst_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/selected_sounds"

create_csv(csv_file_path, wav_folder_path)
gather_files(csv_file_path, wav_folder_path, dst_folder_path)


# Wav to JSON

In [15]:
# CSV to JSON

import pandas as pd
import json
import os

# CSV 파일 읽기
file_name = "synthsoundoutput5.csv"
df = pd.read_csv(file_name)

# A 열의 인덱스를 사용하여 각 행을 JSON 파일로 저장
for index, row in df.iterrows():
    row_data = {key: [value] for key, value in row.items()}  
    a_value = row[df.columns[0]]  
    a_value_without_ext = os.path.splitext(a_value)[0]  # ".wav" 부분을 제거
    json_file_name = f"{a_value_without_ext}.json"  # 파일 이름으로 사용

    # JSON 파일 저장
    with open(json_file_name, 'w', encoding='utf-8') as json_file:
        json.dump(row_data, json_file, ensure_ascii=False, indent=4)

## Train and Test set

In [23]:
import csv
import random


def split_csv(input_csv_path, output_csv_1_path, output_csv_2_path, ratio=0.1):
    # Separate rows based on instrument
    rows_by_instrument = {'lead': [], 'bass': [], 'pluck': []}

    with open(input_csv_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            instrument = row['Instrument']
            if instrument in rows_by_instrument:
                rows_by_instrument[instrument].append(row)

    # Randomly sample rows from each instrument category
    sampled_rows = {'lead': [], 'bass': [], 'pluck': []}
    remaining_rows = {'lead': [], 'bass': [], 'pluck': []}

    for instrument, rows in rows_by_instrument.items():
        sample_size = int(len(rows) * ratio)
        sampled_rows[instrument] = random.sample(rows, sample_size)
        remaining_rows[instrument] = [row for row in rows if row not in sampled_rows[instrument]]

    # Write sampled rows and remaining rows to separate CSV files
    def write_rows_to_csv(output_csv_path, rows_dict):
        with open(output_csv_path, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            csv_writer.writerow(['File_Name', 'Artist', 'Instrument', 'Etc', 'Words'])

            for instrument, rows in rows_dict.items():
                for row in rows:
                    csv_writer.writerow([row['File_Name'], row['Artist'], row['Instrument'], row['Etc'], row['Words']])

    write_rows_to_csv(output_csv_1_path, sampled_rows)
    write_rows_to_csv(output_csv_2_path, remaining_rows)


input_csv_path = 'synthsoundoutput5.csv'
output_csv_1_path = 'stest_10_percent.csv'
output_csv_2_path = 'strain_90_percent.csv'

split_csv(input_csv_path, output_csv_1_path, output_csv_2_path)


#### find train and test wav file

In [22]:
# test
csv_file_path = 'stest_10_percent.csv'
wav_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/soundsprayaiffelsynth"
dst_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/stest"

create_csv(csv_file_path, wav_folder_path)
gather_files(csv_file_path, wav_folder_path, dst_folder_path)

In [18]:
# train
csv_file_path = 'strain_90_percent.csv'
wav_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/soundsprayaiffelsynth"
dst_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/strain"

create_csv(csv_file_path, wav_folder_path)
gather_files(csv_file_path, wav_folder_path, dst_folder_path)

In [24]:
import os
import shutil


def move_wav_files(csv_path, source_folder, destination_folder):
    with open(csv_path, newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.DictReader(csvfile)
        for row in csv_reader:
            file_name = row['File_Name']
            source_file_path = os.path.join(source_folder, file_name)
            destination_file_path = os.path.join(destination_folder, file_name)

            if os.path.exists(source_file_path):
                shutil.move(source_file_path, destination_file_path)
            else:
                print(f"File not found: {source_file_path}")


source_wav_folder_path = "/Users/butterflyeffectleedea/desktop/git/Aiffel/soundsprayaiffelsynth"
train_destination_folder = "strain"
test_destination_folder = "stest"

# Create the destination folders if they don't exist
os.makedirs(train_destination_folder, exist_ok=True)
os.makedirs(test_destination_folder, exist_ok=True)

move_wav_files(output_csv_2_path, source_wav_folder_path, train_destination_folder)
move_wav_files(output_csv_1_path, source_wav_folder_path, test_destination_folder)


In [25]:
# CSV to JSON for train and test

import pandas as pd
import json
import os

# CSV 파일 읽기
file_name = "stest_10_percent.csv"
df = pd.read_csv(file_name)

# 각 행을 JSON 파일로 저장
for index, row in df.iterrows():
    row_data = {key: [value] for key, value in row.items()}  
    a_value = row[df.columns[0]]  
    a_value_without_ext = os.path.splitext(a_value)[0]  # ".wav" 부분을 제거
    json_file_name = f"{a_value_without_ext}.json"  # 파일 이름으로 사용

    # JSON 저장
    with open(json_file_name, 'w', encoding='utf-8') as json_file:
        json.dump(row_data, json_file, ensure_ascii=False, indent=4)

In [26]:
# CSV to JSON

import pandas as pd
import json
import os

# CSV 파일 읽기
file_name = "strain_90_percent.csv"
df = pd.read_csv(file_name)

# A 열의 인덱스를 사용하여 각 행을 JSON 파일로 저장
for index, row in df.iterrows():
    row_data = {key: [value] for key, value in row.items()}  
    a_value = row[df.columns[0]]  
    a_value_without_ext = os.path.splitext(a_value)[0]  # ".wav" 부분을 제거
    json_file_name = f"{a_value_without_ext}.json"  # 파일 이름으로 사용

    # JSON 파일 저장
    with open(json_file_name, 'w', encoding='utf-8') as json_file:
        json.dump(row_data, json_file, ensure_ascii=False, indent=4)